In [4]:
import cv2 as cv
print("Current version of openCV=", cv.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 7
points_per_column = 7
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./pinhole_images/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (points_per_row,points_per_column), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (points_per_row,points_per_column), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("ret: ", ret)
print("K matrix: ", mtx)
print("D distortions: ", dist)
#print("rotation vecs: ", rvecs)
print("translation vecs: ", tvecs)


In [ ]:
img = cv.imread('./pinhole_images/camera_1_jibo_capture_0.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult2.png', dst)

## Fisheye camera calibration

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 7
points_per_column = 7

objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('./fisheye_images/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (points_per_row,points_per_column), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (points_per_row,points_per_column), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

In [ ]:
objpoints=np.array(objpoints, dtype=np.float64)
objpoints = np.reshape(objpoints, (N_OK, 1, points_per_row*points_per_column, 3))
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
cv.destroyAllWindows()

In [ ]:
print("K before: ", K)
rms, _, _, _, _ = \
cv.fisheye.calibrate(objpoints,
                        imgpoints, 
                        gray.shape[::-1], 
                        K, 
                        D, 
                        rvecs, 
                        tvecs,
                        calibration_flags,
                        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))
print("Found " + str(N_OK) + " valid images for calibration")
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

In [ ]:
img = cv.imread('./fisheye_images/camera_0_jibo_capture_0.png')
DIM = img.shape[:2]
print(DIM)
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))

In [ ]:
print("K: ", K)
print("D: ", D)
print("eye: ", np.eye(3))
map1, map2 = cv.fisheye.initUndistortRectifyMap(K, D, np.eye(3), K, DIM, cv2.CV_16SC2)
undistorted_img = cv2.remap(img, map1, map2, interpolation=cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT)
cv.imwrite('calibresult2.png', undistorted_img)

## Stereo

In [5]:
left_calibration_path = './fisheye_images/camera_0_jibo_capture_{}.png'
right_calibration_path = './pinhole_images/camera_1_jibo_capture_{}.png'

In [7]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 7
points_per_column = 7
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in Right image plane.
imgpointsR = [] # 2d points in Left image plane.
imagesR = glob.glob('./pinhole_images/*.png')
imagesL = glob.glob('./fisheye_images/*.png')
for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, (points_per_row,points_per_column), None)
    retR, cornersR = cv.findChessboardCorners(grayR, (points_per_row,points_per_column), None)
    # If found on both, add object points, image points (after refining them)
    if retL == True and retR == True:
        objpoints.append(objp)
        corners2L = cv.cornerSubPix(grayL,cornersL, (11,11), (-1,-1), criteria)
        corners2R = cv.cornerSubPix(grayR,cornersR, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)
        imgpointsR.append(cornersR)
        # Draw and display the corners
        cv.drawChessboardCorners(imgL, (points_per_row,points_per_column), corners2L, retL)
        cv.drawChessboardCorners(imgR, (points_per_row,points_per_column), corners2R, retR)
        cv.imshow('img', imgL)
        cv.waitKey(500)
        cv.imshow('img', imgR)
        cv.waitKey(500)
cv.destroyAllWindows()

In [17]:
#Fisheye calibration
N_OK = len(objpoints)
objpointsL=np.array(objpoints, dtype=np.float64)
objpointsL = np.reshape(objpointsL, (N_OK, 1, points_per_row*points_per_column, 3))
KL = np.zeros((3, 3))
DL = np.zeros((4, 1))
rvecsL = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecsL = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
calibration_flags = cv.fisheye.CALIB_RECOMPUTE_EXTRINSIC
rms, _, _, _, _ = \
cv.fisheye.calibrate(objpointsL,
                        imgpointsL, 
                        grayL.shape[::-1], 
                        KL, 
                        DL, 
                        rvecsL, 
                        tvecsL,
                        calibration_flags,
                        (cv.TERM_CRITERIA_EPS+cv.TERM_CRITERIA_MAX_ITER, 30, 1e-6))
print("Found " + str(N_OK) + " valid images for calibration")
print("K=np.array(" + str(KL.tolist()) + ")")
print("D=np.array(" + str(DL.tolist()) + ")")

Found 20 valid images for calibration
K=np.array([[928.8511307488417, 1.38036641948834, 625.9085651189963], [0.0, 923.1620721974828, 327.6250863531249], [0.0, 0.0, 1.0]])
D=np.array([[-0.14009812701807317], [13.868717767731175], [-186.86952045212655], [895.3455234919691]])


In [19]:
#pinhole camera calibration
#print(objpoints)
ret, KR, DR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, grayR.shape[::-1], None, None)
print("ret: ", ret)
print("K matrix: ", KR)
print("D distortions: ", DR)
#print("rotation vecs: ", rvecsR)
print("translation vecs: ", tvecsR)

ret:  0.24815255257802493
K matrix:  [[1.22924560e+03 0.00000000e+00 6.13328860e+02]
 [0.00000000e+00 1.22572138e+03 3.36227232e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
D distortions:  [[ 2.61443132e-01 -2.42237516e+00 -1.14390298e-03 -1.38365307e-02
   8.99335301e+00]]
translation vecs:  [array([[ 4.88875403],
       [-2.13000921],
       [23.20149881]]), array([[ 5.3900125 ],
       [-2.06512083],
       [23.21531531]]), array([[ 5.63937608],
       [-1.82310004],
       [22.72287733]]), array([[ 5.78507295],
       [-1.63907526],
       [22.4142013 ]]), array([[ 6.54986737],
       [-1.81770001],
       [23.66400455]]), array([[ 6.60864092],
       [-1.87425301],
       [23.73642578]]), array([[ 6.23466202],
       [-2.00757969],
       [24.04539336]]), array([[ 6.40374409],
       [-1.96019044],
       [24.12545593]]), array([[ 6.5992824 ],
       [-1.92109183],
       [24.06976431]]), array([[ 6.63819704],
       [-2.00870099],
       [24.18379193]]), array([[ 6.44839

In [58]:
img = cv.imread(left_calibration_path.format(img_id))
width, height = img.shape[:2]
retval, cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, R, T, E, F = \
 cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, KL ,DL, KR, DR, (width, height),
                                R=R,
                                T=T,
                                E=E,
                                F=F,
                                flags=cv.CALIB_FIX_INTRINSIC
                   )

In [60]:
print("cameraM1: ", cameraMatrix1)
print("distCoeffs1: ", distCoeffs1)
print("cameraM1: ", cameraMatrix2)
print("distCoeffs1: ", distCoeffs2)
print("R: ", R)
print("T: ", T)
print("E: ", E)
print("F: ", F)

cameraM1:  [[928.85113075   1.38036642 625.90856512]
 [  0.         923.1620722  327.62508635]
 [  0.           0.           1.        ]]
distCoeffs1:  [[-1.40098127e-01]
 [ 1.38687178e+01]
 [-1.86869520e+02]
 [ 8.95345523e+02]
 [ 0.00000000e+00]]
cameraM1:  [[1.22924560e+03 0.00000000e+00 6.13328860e+02]
 [0.00000000e+00 1.22572138e+03 3.36227232e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distCoeffs1:  [[ 2.61443132e-01 -2.42237516e+00 -1.14390298e-03 -1.38365307e-02
   8.99335301e+00]]
R:  [[ 0.86465714 -0.44321154 -0.23649853]
 [ 0.48025733  0.59116801  0.6479763 ]
 [-0.14738021 -0.67385749  0.72401323]]
T:  [[ 3.05444374]
 [ 1.33497937]
 [23.35061799]]
E:  [[-11.41105491 -14.70372412 -14.1641044 ]
 [ 20.64044319  -8.29100365  -7.7338445 ]
 [  0.31261954   2.39736768   2.29492782]]
F:  [[-1.42973230e-06 -1.85149811e-06 -1.46927003e-04]
 [ 2.59355136e-06 -1.05209480e-06 -2.18127920e-03]
 [ 5.30220913e-05  1.86075806e-03  1.00000000e+00]]


#### Perform image rectification

In [67]:
img_id = 1

imgL = cv.imread('./fisheye_images/camera_0_jibo_capture_{}.png'.format(img_id))
imgR = cv.imread('./pinhole_images/camera_1_jibo_capture_{}.png'.format(img_id))
img_size = imgL.shape[:2]
print(img_size)

R1 = np.zeros(shape=(3,3))
R2 = np.zeros(shape=(3,3))
P1 = np.zeros(shape=(3,4))
P2 = np.zeros(shape=(3,4))

out = cv.stereoRectify(KL, DL, KR, DR, (width, height), R=R, T=T, 
                       R1=R1, R2=R2, P1=P1, P2=P2,
                      Q=None, flags=cv.CALIB_ZERO_DISPARITY, alpha=-1, newImageSize=(0,0))



(720, 1280)


In [69]:
out

(array([[ 0.98708994, -0.1202754 , -0.10576994],
        [ 0.00675103,  0.69102953, -0.722795  ],
        [ 0.16002461,  0.71274962,  0.68292028]]),
 array([[ 0.93181625,  0.33441779, -0.14100788],
        [-0.12949498, -0.05659726, -0.98996353],
        [-0.33904208,  0.94072392, -0.00943282]]),
 array([[1.07904837e+03, 0.00000000e+00, 9.17322422e+02, 0.00000000e+00],
        [0.00000000e+00, 1.07904837e+03, 2.13618393e+02, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00]]),
 array([[ 1.07904837e+03,  0.00000000e+00,  9.17322422e+02,
          0.00000000e+00],
        [ 0.00000000e+00,  1.07904837e+03,  2.13618393e+02,
         -2.54518932e+04],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00]]),
 array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -9.17322422e+02],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         -2.13618393e+02],
        [ 0.00000000e+00,  0.00000000e+00

In [72]:
print("R1: ", R1)
print("R2: ", R2)
print("P1: ", P1)
print("P2: ", P2)

R1:  [[ 0.98708994 -0.1202754  -0.10576994]
 [ 0.00675103  0.69102953 -0.722795  ]
 [ 0.16002461  0.71274962  0.68292028]]
R2:  [[ 0.93181625  0.33441779 -0.14100788]
 [-0.12949498 -0.05659726 -0.98996353]
 [-0.33904208  0.94072392 -0.00943282]]
P1:  [[1.07904837e+03 0.00000000e+00 9.17322422e+02 0.00000000e+00]
 [0.00000000e+00 1.07904837e+03 2.13618393e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]]
P2:  [[ 1.07904837e+03  0.00000000e+00  9.17322422e+02  0.00000000e+00]
 [ 0.00000000e+00  1.07904837e+03  2.13618393e+02 -2.54518932e+04]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]


NameError: name 'Q' is not defined

In [85]:
print("Applying Undistort\n")
map1x, map1y = cv.initUndistortRectifyMap(KL, DL, R1, P1, (width, height), cv.CV_32FC1)
map2x, map2y = cv.initUndistortRectifyMap(KR, DR, R2, P2, (width, height), cv.CV_32FC1)

print("Undistort complete\n")

for img_id in range(len(imagesR)):
    fnameL = left_calibration_path.format(img_id)
    fnameR = right_calibration_path.format(img_id)
    print("fname: ", fnameL, fnameR)
    imgL = cv.imread(fnameL)
    imgR = cv.imread(fnameR)
    imgU1 = np.zeros((height,width,3), np.uint8)
    imgU1 = cv.remap(imgL, map1x, map1y, cv.INTER_LINEAR, imgU1, cv.BORDER_CONSTANT, 0)
    imgU2 = cv.remap(imgR, map2x, map2y, cv.INTER_LINEAR)
    cv.imshow("imageL", imgL);
    cv.imshow("imageR", imgR);
    cv.imshow("image1L", imgU1);
    cv.imshow("image2R", imgU2);
    k = cv.waitKey(5);
    if(k==27):
        break;

Applying Undistort

Undistort complete

fname:  ./fisheye_images/camera_0_jibo_capture_0.png ./pinhole_images/camera_1_jibo_capture_0.png
fname:  ./fisheye_images/camera_0_jibo_capture_1.png ./pinhole_images/camera_1_jibo_capture_1.png
fname:  ./fisheye_images/camera_0_jibo_capture_2.png ./pinhole_images/camera_1_jibo_capture_2.png
fname:  ./fisheye_images/camera_0_jibo_capture_3.png ./pinhole_images/camera_1_jibo_capture_3.png
fname:  ./fisheye_images/camera_0_jibo_capture_4.png ./pinhole_images/camera_1_jibo_capture_4.png
fname:  ./fisheye_images/camera_0_jibo_capture_5.png ./pinhole_images/camera_1_jibo_capture_5.png
fname:  ./fisheye_images/camera_0_jibo_capture_6.png ./pinhole_images/camera_1_jibo_capture_6.png
fname:  ./fisheye_images/camera_0_jibo_capture_7.png ./pinhole_images/camera_1_jibo_capture_7.png
fname:  ./fisheye_images/camera_0_jibo_capture_8.png ./pinhole_images/camera_1_jibo_capture_8.png
fname:  ./fisheye_images/camera_0_jibo_capture_9.png ./pinhole_images/camera_1

In [86]:
cv.destroyAllWindows()